In [ ]:
library(terra)
library(sf)
library(raster)
library(tmap)
library(tmaptools)
library(lidR)
library(RStoolbox)
library(future)


***LiDAR***

In [ ]:
plan(multisession, workers = 10L)

In [ ]:
las_cat <- catalog("I:\\LiDea II and South\\LiDea II\\LAZ_v2_CGVD2013a")  # Use catalog instead of readLAScatalog
# projection(las_cat) <- "+proj=utm +zone=17 +ellps=GRS80 +datum=NAD83 +units=m +no_defs "
summary(las_cat)
las_check(las_cat)

***Creating a DTM***

In [ ]:
opt_chunk_size(las_cat) <- 500
# plot(las_cat, chunk_pattern = TRUE)
opt_chunk_buffer(las_cat) <- 20
plot(las_cat, chunk_pattern = TRUE)
summary(las_cat)
las_cat@output_options$drivers$Raster$param$overwrite <- TRUE  # overwrite rasters

In [ ]:
opt_output_files(las_cat) <- "I:\\LiDea II and South\\LiDea II\\dtm\\dtm_{XLEFT}_{YBOTTOM}"
dtm <- grid_terrain(las_cat, res = 2, knnidw(k = 10, p = 2), keep_lowest = FALSE)

***Plot DTM***

In [ ]:
tm_shape(dtm) +
  tm_raster(style= "quantile", palette=get_brewer_pal("Greys", plot=FALSE)) +
  tm_layout(legend.outside = TRUE)

***Creating CHM***

In [ ]:
opt_output_files(las_cat) <- "I:\\LiDea II and South\\LiDea II\\norm\\norm_{XLEFT}_{YBOTTOM}"
lasnorm <- normalize_height(las_cat, dtm)

In [ ]:
opt_output_files(lasnorm) <- "I:\\LiDea II and South\\LiDea II\\chm\\chm_{XLEFT}_{YBOTTOM}"
dsm <- grid_canopy(lasnorm, res = 2, pitfree(c(0,2,5,10,15), c(0, 1)))